<a href="https://colab.research.google.com/github/pje1122/pje1122/blob/main/%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0api%ED%99%9C%EC%9A%A9%20%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

국립현대미술관_미술은행 소장품DB 정보
https://www.culture.go.kr/data/openapi/openapiView.do?id=544&category=A&gubun=A#/default/%EC%9A%94%EC%B2%AD%EB%A9%94%EC%8B%9C%EC%A7%80_Get

In [46]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
# 필요한 라이브러리를 임포트
import urllib.request
import datetime
import json
import pandas as pd

# API 호출에 사용할 서비스 키 입력
ServiceKey = "a3f53b21-b546-4020-9202-b3ed861e85a5"

In [57]:
# [CODE 1]
def getKlnReview(url):
  # request 객체 생성
  req = urllib.request.Request(url)
  req.add_header("Content-type", "application/json")
# JSON 형식으로 응답을 받고 싶을 때 아래 주석 제거 (api제공 측 안내)
  req.add_header("Accept", "application/json")


  try :
    # request 에 대한 응답 반환
    response = urllib.request.urlopen(req)

    # HTTP 상태 코드가 200 (OK) 인 경우
    if response.getcode() == 200:
      print("[%s] Url Request Success" % datetime.datetime.now())
      return response.read().decode("utf-8")  # 응답 내용을 utf-8로 디코딩하여 반환합니다.

  # 예외 발생 케이스
  except Exception as e:
    print (e)
    print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
    return None # 실패한 경우 None 을 반환합니다.

In [58]:
# [CODE 2]
# 특정 기간, 특정 국가에 대한 관광객 통계 정보를 요청합니다.
def getTourismStatsItem(numOfRows, pageNo):
  service_url = "http://api.kcisa.kr/API_CCA_141/request"
  parameters = "?serviceKey=" + ServiceKey # 인증키
  parameters += "&numOfRows=" + numOfRows #세션당 요청레코드수
  parameters += "&pageNo=" + pageNo #페이지수

  # 기본URL 에 파라미터를 붙여 요청할 url 생성
  url = service_url + parameters
  # print("url: ", url)

  responseDecode = getKlnReview(url) # [CODE 1] 을 통해 데이터를 반환받습니다.
  #print ("responseDecode: ", responseDecode)

  if responseDecode == None :
    print ("responseDecode == None")
    return None
  else :
    print ("responseDecode == NotNone")
    return  json.loads(responseDecode) #json형식을  파이썬 딕셔너리로 변환

In [59]:
# [CODE 3]
def getTourismStatsService( numOfRows, pageNo):
  jsonResult = [] # json 결과 저장을 위한 리스트
  result = []     # csv 파일 저장을 위한 리스트

  for p in range(1, pageNo+1):
      jsonData = getTourismStatsItem(str(numOfRows), str(p))
      print ("JsonData: ", jsonData)
      for record in range (0,numOfRows):
        t = jsonData['response']['body']['items']['item'][record]['TITLE']
        i = jsonData['response']["body"]["items"]["item"][record]["IMAGE_OBJECT"]
        d = jsonData['response']["body"]["items"]["item"][record]["DESCRIPTION"]
        u = jsonData['response']["body"]["items"]["item"][record]["URL"]

        print ("[ %s : %s : %s]" % (t,d,u))
        print ("-" * 30)
        jsonResult.append({"title": t, "image_object": i, "description": d, "URL" : u})
        result.append([t, i, d, u])

# 수집된 내용을 반환합니다.
  return jsonResult, result


In [60]:
# [CODE 0]
# 메인 함수
def main():
  jsonResult = [] # 최종 JSON 결과 저장을 위한 리스트
  result = []     # 최종 csv 결과 저장을 위한 리스트

  numOfRows = int (input("세션당 요청레코드수"))
  pageNo = int (input("페이지수"))
  jsonResult, result = getTourismStatsService(numOfRows, pageNo) # [CODE 3] 을 통해 데이터 수집

  # 파일 저장 1: json 파일
  with open("/content/drive/My Drive/데이터사이언스/art_data.json", "w", encoding = "utf8") as outfile:
    json.dump(jsonResult, outfile, indent = 4, sort_keys = True, ensure_ascii = False)

  # 파일 저장 2: csv 파일
  columns = ["작품제목", "작품유형", "작품설명","작품정보url"]
  result_df = pd.DataFrame(result, columns = columns)
  result_df.to_csv("/content/drive/My Drive/데이터사이언스/art_data.csv", index=False, encoding="utf-8")

# 메인 함수 진입
if __name__ == '__main__':
  main()

세션당 요청레코드수10
페이지수1
[2024-04-18 07:58:46.687815] Url Request Success
responseDecode == NotNone
JsonData:  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': [{'UCI': 'G706+NMAA04-A.ArtBankDbNew.091125000000065', 'TITLE': '비천(Girl & Cloud)', 'IMAGE_OBJECT': None, 'INGREDIENT': None, 'SIZING': None, 'DESCRIPTION': 'Transformation  Work "Transformation"  is a movement of the images on both sides moving inwards and ', 'URL': ' http://www.artbank.go.kr/home/art/productDetail.do?artId=091125000000065'}, {'UCI': 'G706+NMAA04-A.ArtBankDbNew.200423000034454', 'TITLE': 'enigma-2019', 'IMAGE_OBJECT': '판화', 'INGREDIENT': None, 'SIZING': None, 'DESCRIPTION': '현대판화의 회화적요소를 자신의 손으로, 종이에다 생명을 불어넣는 것과 같은 예술작품을 완성해 가는 판화제작 과정은 저 자신에 있어서 특별한 의미를 가진 즐거움이다.특히 판화기법표현', 'URL': ' http://www.artbank.go.kr/home/art/productDetail.do?artId=200423000034454'}, {'UCI': 'G706+NMAA04-A.ArtBankDbNew.200414000033096', 'TITLE': '태양에 가려 못 봤던 것들_scene Ⅳ', 'IMAGE_OBJECT': '한국화', 'IN